In [1]:
import pandas as pd #Mainly for dataFrame.
import numpy as np #For numeric python like nD arrays.
import matplotlib.pyplot as plt #for plotting graph.
import language_tool_python #for grammer checking.
from spellchecker import SpellChecker #For spell checking.
from sklearn.preprocessing import MinMaxScaler #Scale Score data in 1-100 range.
import re #for regex
import nltk  # to remove stopword 
from nltk.corpus import stopwords # for Stemming propose  
from nltk.stem.porter import PorterStemmer 
from sklearn import linear_model #For linear regression
from sklearn.feature_extraction.text import CountVectorizer # Creating the Bag of Words model 
from sklearn.naive_bayes import GaussianNB # fitting naive bayes to the training set 
from tkinter import *  # For UIX
from tkinter.ttk import *
from tkinter import messagebox


df = pd.read_csv("Processed_DataSet.csv")
df = df.dropna(subset=['Total_Score'])

def wordAndSentenceCount(essay):
#     This funtion will return the score according to the vocabulary
    
    X1 = df[["wordCount"]].dropna()
    Y = df["Total_Score"].dropna()
    
    X2 = df[["sentenceCount"]]
    model1 = linear_model.LinearRegression()
    model2 = linear_model.LinearRegression()
    model1.fit(X1,Y)
    model2.fit(X2,Y)
    
    CountOfWord = len(set(essay.split()))    
    CountOfSentence = len(essay.split('.'))
    preWordScore = model1.predict([[CountOfWord]])
    preSenScore = model2.predict([[CountOfSentence]])
    
    if(preWordScore[0]>100):
        preWordScore = [100];
    if(preSenScore[0]>100):
        preSenScore = [100];
    if(CountOfWord<15):
        preWordScore = [10];
    if(CountOfSentence<=3):
        preSenScore = [10];
        
    return int(preWordScore[0]),int(preSenScore[0])
    

def grammerMistake(essay):
#     This funtion will return score according to grammer errors
    Y = df["Total_Score"].dropna()
    X = df[["grammerMistake"]]
    model = linear_model.LinearRegression()
    model.fit(X,Y)
    
    tool = language_tool_python.LanguageToolPublicAPI('en-US')
    error = tool.check(essay)
    grammerMis = len(error)
    preScore = model.predict([[grammerMis]])
    
    if(preScore[0]>100):
        preScore = [100];
    if(preScore[0]<0):
        preScore = [0]; 
    return int(preScore[0])


def contentBased(essay):
    df1 = pd.read_csv("NLP_Processed_Data.csv")
    df1 = df1.dropna(subset=['Score'])
    corpus = list(df1["Eassy"])
    review = cleanDataNLP(essay)
    
    # Adding the predition part also to get CountVectorizer
    corpus.append(review)
    cv = CountVectorizer(max_features = 1500)  
    X = cv.fit_transform(corpus).toarray()    
    X_Pre = X[-1]  
    Y = df1.iloc[:, 2].values 
    Y=Y.astype('int')
    classifier = GaussianNB(); 
    classifier.fit(X[:-1], Y)
    # predicting test set results 
    y_pred = classifier.predict([X_Pre]) 
    
    return y_pred[0]
    
def cleanDataNLP(essay):
    word = pd.read_excel("word_contraction.xlsx")
    ls1 = list(word["C1"])
    ls2 = list(word["C2"])
    for j in range(len(ls1)):
        essay = essay.replace(ls1[j],ls2[j])

    review = re.sub('[^a-zA-Z]', ' ', essay)  
    # convert all cases to lower cases 
    review = review.lower()  
    # split to array(default delimiter is " ") 
    review = review.split()  
    # creating PorterStemmer object to 
    # take main stem of each word 
    ps = PorterStemmer()  
    # loop for stemming each word 
    # in string array at ith row     
    review = [ps.stem(word) for word in review 
                if not word in set(stopwords.words('english'))]  

    # rejoin all string array elements 
    # to create back into a string 
    review = ' '.join(review)  
    
    return review

def mainFun(opt,essay):
    if len(str(essay))==0:
        messagebox.showerror("Error", "Enter the essay") 
        return;
        
    if len(str(opt))==0:
        messagebox.showerror("Error", "Choose the option for score") 
        return;

    if opt=='1':
        score1 = grammerMistake(essay);
        score2,score3 = wordAndSentenceCount(essay);
        # Score1 is grammer based, score2 is word based, score3 is sentence based
        ans = (score1+score2+score3)//3
        messagebox.showinfo("Result", "Score for given essay is :"+str(ans)) 
        return;
        
    elif opt=='2':
        score = contentBased(essay);
        messagebox.showinfo("Result", "Score for given essay is :"+str(score)) 
        return;
    
    elif opt=='3':
        score1 = grammerMistake(essay);
        score2,score3 = wordAndSentenceCount(essay);
        score4 = contentBased(essay);
        # Score1 is grammer based, score2 is word based, score3 is sentence based, score4 is content based
        gram = (score1+score2+score3)//3
        gram = gram//2
        score4 = score4//2;
        # to get the score out of 50 
        messagebox.showinfo("Result", "Score for given essay is :"+str(score4+gram)) 
        return;

def main():
    master = Tk() 
    master.title("Esaay Auto Grading System")
    Label(master, text = "   ").grid(row = 0, column = 0, sticky = W, padx = 10, pady = 10) 
    Label(master, text = "   ").grid(row = 0, column = 6, sticky = W, padx = 10, pady = 10) 
    Label(master, text = "   ").grid(row = 0, column = 7, sticky = W, padx = 10, pady = 10) 
    Label(master, text = " ").grid(row = 1, column = 1, sticky = W, padx = 10, pady = 10) 
    Label(master, text = " ").grid(row = 3, column = 3, sticky = W, padx = 10, pady = 10)  

    Label(master, text = "Welcome to Auto",font=("Arial", 20) ).grid(row = 2, column = 4, sticky = W, padx = 5, pady = 10) 
    Label(master, text = " Essay Grading System",font=("Arial", 20) ).grid(row = 2, column = 5, sticky = W, padx = 0, pady = 10) 
    Label(master, text = "Enter your Essay:",font=("Arial", 16) ).grid(row = 4, column = 4, sticky = W, padx = 10, pady = 10) 
    e1 = Entry(master) 
    e1.grid(row = 4, column = 5, padx = 10, pady = 10) 
    v1 = StringVar(master, "")

    Radiobutton(master, text = "Grammer based grading", variable = v1, value = "1").grid(sticky = W,row = 5, column = 4, padx = 10, pady = 10) 
    Radiobutton(master, text = "Content based grading", variable = v1, value = "2").grid(sticky = W,row = 6, column = 4, padx = 10, pady = 10) 
    Radiobutton(master, text = "Both grading", variable = v1, value = "3").grid(sticky = W,row = 7, column = 4, padx = 10, pady = 10) 
    printScore = v1.get()
    btn = Button(master, text = 'Check Score',command = lambda: mainFun(v1.get(),e1.get())).grid(row = 9, column = 5, sticky = W, padx = 10, pady = 10) 

    Label(master, text = "   ").grid(row = 11, column = 0, sticky = W, padx = 10, pady = 10) 
    mainloop() 
    
main()


In [54]:
from tkinter import * 
from tkinter.ttk import *

def mainFun(opt,essay):
    if len(str(essay))==0:
        messagebox.showerror("Error", "Enter the essay") 
        return;
        
    if len(str(opt))==0:
        messagebox.showerror("Error", "Choose the option for score") 
        return;

    if opt=='1':
        score1 = grammerMistake(essay);
        score2,score3 = wordAndSentenceCount(essay);
        # Score1 is grammer based, score2 is word based, score3 is sentence based
        ans = (score1+score2+score3)//3
        messagebox.showinfo("Result", "Score for given essay is :"+str(ans)) 
        return;
        
    elif opt=='2':
        score = contentBased(essay);
        messagebox.showinfo("Result", "Score for given essay is :"+str(score)) 
        return;
    
    elif opt=='3':
        score1 = grammerMistake(essay);
        score2,score3 = wordAndSentenceCount(essay);
        score4 = contentBased(essay);
        # Score1 is grammer based, score2 is word based, score3 is sentence based, score4 is content based
        gram = (score1+score2+score3)//3
        gram = gram//2
        score4 = score4//2;
        # to get the score out of 50 
        messagebox.showinfo("Result", "Score for given essay is :"+str(score4+gram)) 
        return;

def main():
    master = Tk() 

    Label(master, text = "   ").grid(row = 0, column = 0, sticky = W, padx = 10, pady = 10) 
    Label(master, text = "   ").grid(row = 0, column = 6, sticky = W, padx = 10, pady = 10) 
    Label(master, text = "   ").grid(row = 0, column = 7, sticky = W, padx = 10, pady = 10) 
    Label(master, text = " ").grid(row = 1, column = 1, sticky = W, padx = 10, pady = 10) 
    Label(master, text = " ").grid(row = 3, column = 3, sticky = W, padx = 10, pady = 10)  

    Label(master, text = "Welcome to Auto",font=("Arial", 20) ).grid(row = 2, column = 4, sticky = W, padx = 5, pady = 10) 
    Label(master, text = " Essay Grading System",font=("Arial", 20) ).grid(row = 2, column = 5, sticky = W, padx = 0, pady = 10) 
    Label(master, text = "Enter your Essay:",font=("Arial", 16) ).grid(row = 4, column = 4, sticky = W, padx = 10, pady = 10) 
    e1 = Entry(master) 
    e1.grid(row = 4, column = 5, padx = 10, pady = 10) 
    v1 = StringVar(master, "")

    Radiobutton(master, text = "Grammer based grading", variable = v1, value = "1").grid(sticky = W,row = 5, column = 4, padx = 10, pady = 10) 
    Radiobutton(master, text = "Content based grading", variable = v1, value = "2").grid(sticky = W,row = 6, column = 4, padx = 10, pady = 10) 
    Radiobutton(master, text = "Both grading", variable = v1, value = "3").grid(sticky = W,row = 7, column = 4, padx = 10, pady = 10) 
    printScore = v1.get()
    btn = Button(master, text = 'Check Score',command = lambda: mainFun(v1.get(),e1.get())).grid(row = 9, column = 5, sticky = W, padx = 10, pady = 10) 

    Label(master, text = "   ").grid(row = 11, column = 0, sticky = W, padx = 10, pady = 10) 
    mainloop() 
    
main()

In [49]:

from tkinter import messagebox 

messagebox.showinfo("showinfo", "Information") 
messagebox.showwarning("showwarning", "Warning") 
  
messagebox.showerror("showerror", "Error") 
  
messagebox.askquestion("askquestion", "Are you sure?") 
  
messagebox.askokcancel("askokcancel", "Want to continue?") 
  
messagebox.askyesno("askyesno", "Find the value?") 
messagebox.askretrycancel("askretrycancel", "Try again?") 

TclError: can't invoke "grab" command: application has been destroyed